# Assignment 2 - Gesture Based UI

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from PIL import Image
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.10.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
# Define constants
IMAGE_SIZE = (128, 128)
BATCH_SIZE = 32
DATASET_PATH = "hagridset"
SEED = 399177

In [3]:
train_dataset = tf.keras.utils.image_dataset_from_directory(DATASET_PATH,
                                                            shuffle=True,
                                                            image_size=IMAGE_SIZE, validation_split=0.3
                                                           , subset='training', label_mode='categorical', 
                                                           seed=SEED, batch_size=BATCH_SIZE)

validation_and_test_dataset = tf.keras.utils.image_dataset_from_directory(DATASET_PATH,
                                                            shuffle=True,
                                                            image_size=IMAGE_SIZE, validation_split=0.3
                                                           , subset='validation',label_mode='categorical',
                                                           seed=SEED, batch_size=BATCH_SIZE)

val_batches = tf.data.experimental.cardinality(validation_and_test_dataset)

test_dataset = validation_and_test_dataset.take((2*val_batches) // 3)
validation_dataset = validation_and_test_dataset.skip((2*val_batches) // 3)


Found 125912 files belonging to 18 classes.
Using 88139 files for training.
Found 125912 files belonging to 18 classes.
Using 37773 files for validation.


In [4]:
# Get the number of classes from the data generator
NUM_CLASSES = len(train_dataset.class_names)
print("Classes: ",NUM_CLASSES)

Classes:  18


In [5]:
data_augmentation_layers = tf.keras.Sequential([
    # layers.RandomFlip("horizontal"),
    # layers.RandomRotation(0.1),
])

In [6]:
# CNN model from scratch
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1],3)))
model.add(data_augmentation_layers)
model.add(layers.Rescaling(1./255))
model.add(layers.Conv2D(16, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(32, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(NUM_CLASSES, activation='softmax'))

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [7]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [8]:
model.fit(train_dataset,  validation_data=(validation_dataset), epochs=10, callbacks=[callback])

Epoch 1/10
2755/2755 [==============================] - 37s 13ms/step - loss: 2.0995 - accuracy: 0.3275 - val_loss: 1.5349 - val_accuracy: 0.5009
Epoch 2/10
2755/2755 [==============================] - 32s 12ms/step - loss: 1.2556 - accuracy: 0.5956 - val_loss: 1.2187 - val_accuracy: 0.6056
Epoch 3/10
2755/2755 [==============================] - 31s 11ms/step - loss: 0.8928 - accuracy: 0.7101 - val_loss: 1.1490 - val_accuracy: 0.6420
Epoch 4/10
2755/2755 [==============================] - 31s 11ms/step - loss: 0.6536 - accuracy: 0.7862 - val_loss: 1.3003 - val_accuracy: 0.6392
Epoch 5/10
2755/2755 [==============================] - 30s 11ms/step - loss: 0.4739 - accuracy: 0.8437 - val_loss: 1.5476 - val_accuracy: 0.6421
Epoch 6/10
2755/2755 [==============================] - 30s 11ms/step - loss: 0.3481 - accuracy: 0.8824 - val_loss: 1.7294 - val_accuracy: 0.6354
Epoch 7/10
2755/2755 [==============================] - 32s 11ms/step - loss: 0.2715 - accuracy: 0.9073 - val_loss: 2.0034 -

64x64: With the contrast at 0.5 epoch tens accuracy was 0.5199 </br>
64x64: with just flipping and rotation it had an accuracy of 0.5697</br>



In [9]:
model.evaluate(test_dataset, return_dict=True)

787/787 [==============================] - 5s 6ms/step - loss: 1.1444 - accuracy: 0.6447


{'loss': 1.1443893909454346, 'accuracy': 0.6446950435638428}

In [10]:
class_names = train_dataset.class_names


['call',
 'dislike',
 'fist',
 'four',
 'like',
 'mute',
 'ok',
 'one',
 'palm',
 'peace',
 'peace_inverted',
 'rock',
 'stop',
 'stop_inverted',
 'three',
 'three2',
 'two_up',
 'two_up_inverted']

In [11]:
image = np.array(Image.open("MyHandStop.jpg").resize((IMAGE_SIZE[0],IMAGE_SIZE[1])))
image = image.reshape(1,IMAGE_SIZE[0],IMAGE_SIZE[1],3)

ans = model.predict(image)
pd.DataFrame(ans, columns=class_names).idxmax(axis=1)

1/1 [==============================] - 0s 78ms/step


0    stop
dtype: object

# Transfer Learning

In [12]:
inputs = tf.keras.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1],3))

In [13]:
base_model = tf.keras.applications.VGG16(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1],3),
    include_top=False)
base_model.trainable = False
x = data_augmentation_layers(inputs)
x = tf.keras.applications.xception.preprocess_input(x)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(NUM_CLASSES)(x)
model = tf.keras.Model(inputs, outputs)

In [14]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [15]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [16]:
model.fit(train_dataset,  validation_data=(validation_dataset), epochs=10, callbacks=[callback])

Epoch 1/10
2755/2755 [==============================] - 79s 28ms/step - loss: 2.0949 - accuracy: 0.3710 - val_loss: 1.7748 - val_accuracy: 0.4699
Epoch 2/10
2755/2755 [==============================] - 75s 27ms/step - loss: 1.7500 - accuracy: 0.4600 - val_loss: 1.6227 - val_accuracy: 0.5110
Epoch 3/10
2755/2755 [==============================] - 76s 27ms/step - loss: 1.6737 - accuracy: 0.4800 - val_loss: 1.5605 - val_accuracy: 0.5261
Epoch 4/10
2755/2755 [==============================] - 74s 27ms/step - loss: 1.6384 - accuracy: 0.4890 - val_loss: 1.5236 - val_accuracy: 0.5365
Epoch 5/10
2755/2755 [==============================] - 75s 27ms/step - loss: 1.6214 - accuracy: 0.4916 - val_loss: 1.5028 - val_accuracy: 0.5380
Epoch 6/10
2755/2755 [==============================] - 74s 27ms/step - loss: 1.6119 - accuracy: 0.4940 - val_loss: 1.4903 - val_accuracy: 0.5417
Epoch 7/10
2755/2755 [==============================] - 75s 27ms/step - loss: 1.6011 - accuracy: 0.4978 - val_loss: 1.4748 -

In [17]:
model.evaluate(test_dataset, return_dict=True)

787/787 [==============================] - 18s 22ms/step - loss: 1.4532 - accuracy: 0.5502


{'loss': 1.4532345533370972, 'accuracy': 0.5502303242683411}

epoch ten transfer learning: val_accuracy: 0.3462